In [1]:
import pandas as pd
import os
import fnmatch
import re
import multiprocessing as mp
import xlrd
import pickle
import time
from readExcelFiles import *
from openpyxl import load_workbook
import dateutil.parser as dparser
import dateutil
import locale
import datetime



In [3]:
def get_sheetnames_xlsx(filepath):
    wb = load_workbook(filepath, read_only=True, keep_links=False)
    print(wb)
    return wb.sheetnames

# Let's set a non-US locale

Unidades=['Eco06','Eco14','Eco18','Eco07']

BuscarPages={'RM. TRACTO 14 P  70AD2V':'Eco14','RM TRACTO 18 P  882ER4':'Eco18','RM. TRACTO 07 P  103ED7':'Eco07','RM TRACTO 06 P  63AP1G':'Eco06'}

Buscar_sensores={"Odometro":"Kilometraje","Sensor 1":"TANQUE 1","Sensor 2":"TANQUE 2", "Total Fuell Canbus Inf":"Total fuel used"}

Buscar_canbus={"Total Fuell Canbus Inf":"Total fuel used","Temperatura del motor °C":"Temperatura del motor","RPM":"RPM"}

Extract_info_sheetsLook={'Eco14':'RM. TRACTO 14 P  70AD2V - 2','Eco07':'RM. TRACTO 07 P  103ED7 - 2','Eco18':'RM TRACTO 18 P  882ER4 - 2','Eco06':'RM TRACTO 06 P  63AP1G - 2'}

unidadesLitrs_df={'Eco14':{"Total Fuell Canbus Inf":[],"Odometro":[],"Sensor 1":[],"Sensor 2":[]},'Eco07':{"Total Fuell Canbus Inf":[],"Odometro":[],"Sensor 1":[],"Sensor 2":[]},'Eco18':{"Total Fuell Canbus Inf":[],"Odometro":[],"Sensor 1":[],"Sensor 2":[]},'Eco06':{"Total Fuell Canbus Inf":[],"Odometro":[],"Sensor 1":[],"Sensor 2":[]}}

unidadescanbus_df={'Eco18':{"Total Fuell Canbus Inf":[],"Temperatura del motor °C":[],"RPM":[]},'Eco14':{"Total Fuell Canbus Inf":[],"Temperatura del motor °C":[],"RPM":[]},'Eco07':{"Total Fuell Canbus Inf":[],"Temperatura del motor °C":[],"RPM":[]},'Eco06':{"Total Fuell Canbus Inf":[],"Temperatura del motor °C":[],"RPM":[]}}


df = pd.DataFrame([], columns = ["Odometro","Sensor 1","Sensor 2","file"])
df_c = pd.DataFrame([], columns = ["Total Fuell Canbus Inf","Temperatura del motor °C","RPM"])

unidadesLitrs_df1={'Eco14':df,'Eco07':df,'Eco18':df,'Eco06':df}

path_to_files="/home/rokdrigo/GitClone/Sensores2/Sensores/TEMP/2021 Octubre/"
os.chdir(path_to_files)
#path_engine='sqlite:////home/rokdrigo/GitClone/Combustible2/Sensores/sqlalchemy.db'

ListFilesLoad=([file for file in os.listdir('.') if fnmatch.fnmatch(file, '*.xlsx')])
#ListFilesUnidad=([file for file in ListFilesLoad if re.search(self.economico,file)])

for file in ListFilesLoad:
    litros_df=[]
    print(file)      
    wb=load_workbook(path_to_files+file, read_only=True, keep_links=False)
    for sheet_name in wb.sheetnames:
        if sheet_name in BuscarPages.keys():
            sheet = wb[sheet_name]
            unidad=BuscarPages[sheet_name]
            #print(sheet_name)
            cellA4=sheet.cell(row=4, column=1).value
            print(cellA4)
            print(sheet_name)
            for k in  Buscar_sensores:
                if Buscar_sensores[k] in cellA4:
                    print(k)
                    print( Buscar_sensores[k])
                    pag_extract=wb[Extract_info_sheetsLook[unidad]]
                    dates_list=[] 
                    date_string=pag_extract.cell(row=3, column=1).value
                    if isinstance(date_string,str):
                        locale.setlocale(locale.LC_ALL,'es_MX')
                        match_str = re.search(r'\d{2}-\w{3}-\d{4}', date_string)
                        if match_str.group():
                            res = datetime.datetime.strptime(match_str.group(),'%d-%b-%Y').date()
                            print(res)
                            sheet_info=pd.read_excel(path_to_files+file,usecols='A:B',skiprows=1,sheet_name=Extract_info_sheetsLook[unidad], engine='openpyxl')
                            sheet_info['file'+k]=file
                            sheet_info['mont']=pd.to_datetime(res).to_period('M')
                            sheet_info=sheet_info.dropna()
                            periods=len(sheet_info.index)
                            datelist = pd.date_range(res, periods=(periods),freq='5min').tolist()
                            sheet_info['index_2']=datelist
                            sheet_info.set_index('index_2', inplace=True)
                            sheet_info = sheet_info.rename(columns={sheet_info.columns[1]:k})
                            unidadesLitrs_df[unidad][k].append(sheet_info)        
                            
for unidad in unidadesLitrs_df:
    print(unidad)
    for sensor in unidadesLitrs_df[unidad]:
        if isinstance(unidadesLitrs_df[unidad][sensor],list) and len(unidadesLitrs_df[unidad][sensor])>0:
            unidadesLitrs_df[unidad][sensor]=pd.concat(unidadesLitrs_df[unidad][sensor])
        else:
            print('Sin sensores que concatenar')
            
for unidad in unidadesLitrs_df:
    print('Unidad concatenando {}'.format(unidad))
    if  'Odometro' in unidadesLitrs_df[unidad].keys():
        if isinstance(unidadesLitrs_df[unidad]['Odometro'], pd.DataFrame):
            if len(unidadesLitrs_df[unidad]['Odometro'])>1:
                o1,s1,s2= len(unidadesLitrs_df[unidad]['Odometro']),len(unidadesLitrs_df[unidad]['Sensor 1']),len(unidadesLitrs_df[unidad]['Sensor 2'])  
                tf = len(unidadesLitrs_df[unidad]["Total Fuell Canbus Inf"])
                print('Long Odometro {}, Long Sensor 1 {}, Long Sensor 2 {}, tf {}'.format(o1,s1,s2,tf))
              
                if o1 == s1 and s2 == tf:
                    if o1 == tf:
                        df=pd.concat([unidadesLitrs_df[unidad]['Odometro'],unidadesLitrs_df[unidad]['Sensor 1'],unidadesLitrs_df[unidad]['Sensor 2'],unidadesLitrs_df[unidad]["Total Fuell Canbus Inf"]],axis=1)
                        df = df.loc[:,~df.columns.duplicated()]
                        unidadesLitrs_df[unidad]=df
                    else:
                        print('**********'*50)
                        print('Longitudes a concatenar no son iguales error en los archivos de la unidad')
                        print('unidad {},{}'.format(unidad,'Odometro'))
                
                        print('Long Odometro {}, Long Sensor 1 {}, Long Sensor 2 {}, tf {}'.format(o1,s1,s2,tf))
                        

for unidad in unidadesLitrs_df:
    if isinstance(unidadesLitrs_df[unidad], pd.DataFrame):
        names=unidadesLitrs_df[unidad].columns
        if  'Sensor 1' in names and 'Sensor 2' in names:
            output=unidadesLitrs_df[unidad][['Sensor 1','Sensor 2','Odometro','Total Fuell Canbus Inf']].copy()
            output.index.names=['Tiempo  d/m/a hh:mm:ss']
            # shift column 'Name' to first position
            Time = output.index
            # insert column using insert(position,column_name,
            # first_column) function
            output.insert(0, '', Time)
            output[''] = pd.to_datetime(output['']).dt.time
            output.index=output.index.strftime('%d/%m/%Y')  
            outputCanbus=output[['','Total Fuell Canbus Inf']].copy()
            month=datetime.datetime.strptime(output.index[0],'%d/%m/%Y').strftime('%B')
            year=datetime.datetime.strptime(output.index[0],'%d/%m/%Y').strftime('%Y')
            output[['','Sensor 1','Sensor 2','Odometro']].to_csv(path_to_files+month+year+unidad+'.csv')
            outputCanbus.to_csv(path_to_files+month+year+unidad+'Canbus'+'.csv')
            print('Succed files {}'.format(path_to_files+month+year+unidad))

            


Tanque_2-20211101-1100.xlsx
Las lecturas del sensor "TD TRACTO 14 TANQUE 2 LLS"
RM. TRACTO 14 P  70AD2V
Sensor 2
TANQUE 2
2021-10-01


/home/rokdrigo/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Las lecturas del sensor "TD TRACTO 07 TANQUE 2 LLS"
RM. TRACTO 07 P  103ED7
Sensor 2
TANQUE 2
2021-10-01


/home/rokdrigo/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Las lecturas del sensor "TRACTO 18  TANQUE 2 LLS"
RM TRACTO 18 P  882ER4
Sensor 2
TANQUE 2
2021-10-01


...

Tanque_1_BT_del_15_al_28_octubre-20211103-2029.xlsx
Las lecturas del sensor "TRACTO 18 TANQUE 1 BT"
RM TRACTO 18 P  882ER4
Sensor 1
TANQUE 1
2021-10-15


/home/rokdrigo/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Tanque_1_BT_del_1_al_14_octubre-20211103-2027.xlsx
Las lecturas del sensor "TRACTO 18 TANQUE 1 BT"
RM TRACTO 18 P  882ER4
Sensor 1
TANQUE 1
2021-10-01
Tanque1-20211101-2100.xlsx
Las lecturas del sensor "TD TRACTO 14 TANQUE 1 LLS"
RM. TRACTO 14 P  70AD2V
Sensor 1
TANQUE 1
2021-10-01
Las lecturas del sensor "TD TRACTO 07 TANQUE 1 LLS"
RM. TRACTO 07 P  103ED7
Sensor 1
TANQUE 1
2021-10-01
Tanque_1_dek_29_al_31_de_octubre-20211103-2055.xlsx
Las lecturas del sensor "TRACTO 18 TANQUE 1 BT"
RM TRACTO 18 P  882ER4
Sensor 1
TANQUE 1
2021-10-29


/home/rokdrigo/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Eco14
Eco07
Eco18
Eco06
Sin sensores que concatenar
Sin sensores que concatenar
Sin sensores que concatenar
Sin sensores que concatenar
Unidad concatenando Eco14
Long Odometro 8940, Long Sensor 1 8940, Long Sensor 2 8940, tf 8940
Unidad concatenando Eco07
Long Odometro 8940, Long Sensor 1 8940, Long Sensor 2 8940, tf 8940
Unidad concatenando Eco18
Long Odometro 8940, Long Sensor 1 8940, Long Sensor 2 8940, tf 8940
Unidad concatenando Eco06
Succed files /home/rokdrigo/GitClone/Sensores2/Sensores/TEMP/2021 Octubre/octubre2021Eco14
Succed files /home/rokdrigo/GitClone/Sensores2/Sensores/TEMP/2021 Octubre/octubre2021Eco07
Succed files /home/rokdrigo/GitClone/Sensores2/Sensores/TEMP/2021 Octubre/octubre2021Eco18


In [ ]:
unidad='Eco18'
#df=pd.concat([unidadesLitrs_df[unidad]['Odometro'],unidadesLitrs_df[unidad]['Sensor 1'],unidadesLitrs_df[unidad]['Sensor 2'],unidadesLitrs_df[unidad]["Total Fuell Canbus Inf"]],axis=1)
print(len(unidadesLitrs_df[unidad]['Odometro']))
print(len(unidadesLitrs_df[unidad]['Sensor 1']))
print(len(unidadesLitrs_df[unidad]['Sensor 2']))
print(len(unidadesLitrs_df[unidad]["Total Fuell Canbus Inf"]))

In [ ]:
unidadesLitrs_df[unidad]['Sensor 1']

In [ ]:
for unidad in unidadesLitrs_df:
    if isinstance(unidadesLitrs_df[unidad], pd.DataFrame):
        names=unidadesLitrs_df[unidad].columns
        if  'Sensor 1' in names and 'Sensor 2' in names:
            output=unidadesLitrs_df[unidad][['Sensor 1','Sensor 2','Odometro','Total Fuell Canbus Inf']].copy()
            output.index.names=['Tiempo  d/m/a hh:mm:ss']
            # shift column 'Name' to first position
            Time = output.index
            # insert column using insert(position,column_name,
            # first_column) function
            output.insert(0, '', Time)
            output[''] = pd.to_datetime(output['']).dt.time
            output.index=output.index.strftime('%d/%m/%Y')  
            outputCanbus=output[['','Total Fuell Canbus Inf']].copy()
            month=datetime.datetime.strptime(output.index[0],'%d/%m/%Y').strftime('%B')
            year=datetime.datetime.strptime(output.index[0],'%d/%m/%Y').strftime('%Y')
            output[['','Sensor 1','Sensor 2','Odometro']].to_csv(path_to_files+month+year+unidad+'.csv')
            outputCanbus.to_csv(path_to_files+month+year+unidad+'Canbus'+'.csv')

            

In [ ]:
unidadesLitrs_df['Eco14'].keys()